In [1]:
# Estimate of segment volumes (upper & lower)
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import TEF_Variables as tef

In [2]:
# Load fields
Qsg = 700
hs = 30
State = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf' + str(format(Qsg,'03d')) + '_rot/state_' + str(format(hs,'03d')) + '.nc')
Grid = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf' + str(format(Qsg,'03d')) + '_rot/grid_' + str(format(hs,'03d')) + '.nc')
# Confine to the range of fjord
state = State.isel(X=range(120), Xp1=range(121), Y=range(30,39), T=range(1,len(State.T)))
grid = Grid.isel(X=range(120), Xp1=range(121), Y=range(30,39))

ot = state.T.data # Ocean Time in seconds
x = grid.X.data

S = state.S.data # Salinity
U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2 # Along-channel velocity
#drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
#dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
#HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
HFacC = grid.HFacC.data
CV = np.broadcast_to(grid.drF.data[:, np.newaxis, np.newaxis], HFacC.shape) * np.broadcast_to(grid.rA.data[np.newaxis, :, :], HFacC.shape) * HFacC # Cell Volume
CV.shape

(50, 9, 120)

In [3]:
uzx = U[274:,:,:,48:72].mean(axis=(0,2))

v1 = np.empty(uzx.shape[1])
v2 = np.empty(uzx.shape[1])
for i in range(uzx.shape[1]):
    
    l = np.where(uzx[:,i] < 0)[-1][0]
    v1[i] = CV[:l,:,i+48].sum()
    v2[i] = CV[l:,:,i+48].sum()
    

In [4]:
v1.sum() / CV[:,:,48:72].sum()

0.15842973548009018